In [1]:
!pip install tensorflow==2.3.1
!pip install q keras==2.4.3
!pip install numpy==1.18.1
!pip install pandas==1.0.1
!pip install esprima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, precision_recall_curve, classification_report, confusion_matrix
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Conv1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping,Callback,CSVLogger
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Activation, Dense, Reshape, Input
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential, load_model
from keras.preprocessing.text import Tokenizer
from keras.models import Model , clone_model
from keras.losses import BinaryCrossentropy
from keras.backend import clear_session
from keras.initializers import Constant
from nltk.tokenize import word_tokenize
from threading import current_thread
from keras.utils import plot_model
from sklearn.utils import shuffle
from keras.optimizers import Adam
from keras.utils import Sequence
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from keras import backend as K
from functools import partial
from numpy import asarray
import concurrent.futures
from keras import layers
import matplotlib as mpl
from keras import Model
from numpy import save
from numpy import load
import seaborn as sns
import esprima as esp
from time import time
import pandas as pd
import numpy as np
import tempfile
import logging
import urllib
import random
import pickle
import pydot
import torch
import math
import html
import nltk
import os
import re
import gc
np.random.seed(123)
plt.style.use('ggplot')
# mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
par_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Hiden_Co"
model_info_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Results/Model info"
results_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Results"
data_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/DataSets"

In [5]:
class GarbageCollectorCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [6]:
def save_object(obj, filename,path):
    """
    _ INPUT (obj) THE OBJECT WE NEED SAVW IT (filename) THE NAME OF OBJECT
    """
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()

In [7]:
def load_object(filename,path):
    """
    _ INPUT THE NAME OF OBJECT WE NEED LOAD IT
    """
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

In [8]:
def Pad_Sequences(scripts,fit = True,creat=True):
    if fit:# and creat:
        tokenizer = Tokenizer(num_words=num_words)
        tokenizer.fit_on_texts(scripts)
        save_object(tokenizer, "Pad_Sequences_tokenizer",results_path)
    # elif fit and not creat:
    #     tokenizer = load_object("Pad_Sequences_tokenizer",results_path)
    #     tokenizer.fit_on_texts(scripts)
    #     save_object(tokenizer, "Pad_Sequences_tokenizer",results_path)
    else :
        tokenizer = load_object("Pad_Sequences_tokenizer",results_path)
    X = tokenizer.texts_to_sequences(scripts)
    X = pad_sequences(X, padding='post', maxlen=maxlen)
    return X

In [9]:
def chunking_data(Data,test_size=0.20):
    """
    _INPUT (Data) CHUNK OF DATASET (test_size=0.20) SIZE OF SPLITNG
    _OUTPUT (X_train , y_train , X_test , y_test)
    """
    train_data, test_data = train_test_split(Data, test_size=0.2,random_state=42,shuffle=False)
    X_test = test_data[test_data.columns[-2]]
    y_test = test_data[test_data.columns[-1]]
    X_train = train_data[train_data.columns[-2]]
    y_train = train_data[train_data.columns[-1]]
    return X_train , y_train , X_test , y_test

In [10]:
def create_model(shape=None,without_first_layer=False):
    """
    _INPUT (without_first_layer) IF WE WANT TO CREAT MODEL WITHOUT FIRST LSTM LAYER (shape) IF THE PREVIOS PAREMETER IS TRUE WE NEED DIFINE THE SHAPE FOR INPUT LAYER
    _OUTPUT THE MODEL 
    """
    clear_session()
    model = Sequential() 
    if without_first_layer:
        name_model = 'without_biLSTM_layers_weights.best.hdf5'
        model.add(layers.InputLayer(input_shape=shape))
        # model.add(Bidirectional(LSTM(units=50, input_shape=(None, 50), return_sequences=True)))
    else:
        name_model = 'with_biLSTM_layers_weights.best.hdf5'
        model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
        model.add(Bidirectional(LSTM(units=50, input_shape=(None, 50), return_sequences=True))) 
        model.add(Bidirectional(LSTM(units=50, input_shape=(None, 50), return_sequences=True)))
    # TextCNN with 4 conv layers
    model.add(Conv1D(128, 7, activation='tanh', input_shape=(None, 32)))
    model.add(Conv1D(128, 15, activation='tanh'))
    model.add(Conv1D(128, 25, activation='tanh'))
    model.add(Conv1D(128, 35, activation='tanh'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    model.load_weights(os.path.join(model_info_path,name_model))
    plot_model(model, to_file=os.path.join(model_info_path,name_model[:-10]+'.png'), show_shapes=True, show_layer_names=True)
    # model.summary()
    return model

In [11]:
vocab_size = 500000
num_words = 400000
maxlen = 2048
embedding_dim = 512

model = create_model()

In [12]:
for i,df_iterator in tqdm(enumerate(pd.read_csv(os.path.join(data_path,"XSS_dataset_payload.csv"),chunksize=2000))):
  df_iterator = df_iterator.mask(df_iterator.eq(np.nan)).dropna()
  X_train , y_train , X_test , y_test  = chunking_data(df_iterator)
  X_train = Pad_Sequences(X_train,fit = True)
  X_test = Pad_Sequences(X_test,fit=False) # ,creat=False

  sub_model_1 = Model(inputs=model.input,outputs=model.layers[1].output)
  sub_model_2 = Model(inputs=model.input,outputs=model.layers[2].output)

  X_train_1 = sub_model_1.predict(X_train)
  X_train_2 = sub_model_1.predict(X_train)

  X_test_1 = sub_model_1.predict(X_test)
  X_test_2 = sub_model_2.predict(X_test)

  Train_Dynamic_word_embeding = []
  for j in range(X_train_1.shape[0]):
    temp = np.concatenate((X_train_1[i], X_train_2[i]), axis=1)
    Train_Dynamic_word_embeding.append(np.concatenate((temp, np.reshape(X_train[i],(X_train[i].shape[0],1))), axis=1))
  
  Test_Dynamic_word_embeding = []
  for j in range(X_test_1.shape[0]):
    temp = np.concatenate((X_test_1[i], X_test_2[i]), axis=1)
    Test_Dynamic_word_embeding.append(np.concatenate((temp, np.reshape(X_test[i],(X_test[i].shape[0],1))), axis=1))
    

  save_object(np.array(Train_Dynamic_word_embeding), "X_train_co_hiden_for_chunk_"+str(i),par_path)
  save_object(y_train, "y_train_co_hiden_for_chunk_"+str(i),par_path)
  
  save_object(np.array(Test_Dynamic_word_embeding), "X_test_co_hiden_for_chunk_"+str(i),par_path)
  save_object(y_test, "y_test_co_hiden_for_chunk_"+str(i),par_path)

0it [00:00, ?it/s]

In [12]:
model.save(os.path.join(model_info_path,"model1.h5"))